## Libs

In [1]:
import sqlite3
import pandas as pd

## Modulo criado com um dict de querys SQlite

In [2]:
from querys import querys_sql

# Conexão com o banco de dados

In [3]:
conn = sqlite3.connect('movies.db')
cursor = conn.cursor()

# criação dos DataFrames com a execução a query SQlite

In [4]:
df_top_rated = pd.read_sql_query(querys_sql.get("top_rated_query"), conn)
df_genre_popularity = pd.read_sql_query(querys_sql.get("genre_popularity_query"), conn)
df_country_rating = pd.read_sql_query(querys_sql.get("country_rating_query"), conn)

df_movie_characters_with_actors = pd.read_sql_query(querys_sql.get("movie_characters_with_actors"), conn)
df_movie_production_companies = pd.read_sql_query(querys_sql.get("movie_production_companies"), conn)
df_most_expensive_movies = pd.read_sql_query(querys_sql.get("most_expensive_movies"), conn)
df_available_languages_by_movie = pd.read_sql_query(querys_sql.get("available_languages_by_movie"), conn)


### Algumas checagens de dados

In [5]:
print(df_country_rating.head())
len(df_country_rating)

  country_name  avg_rating  movie_count
0      Austria    7.083333            6
1    Argentina    6.988889            9
2       Sweden    6.910526           19
3  New Zealand    6.732143           28
4      Iceland    6.716667            6


36

In [6]:
print(df_top_rated.head())
len(df_top_rated)

                   title  vote_average  popularity release_date
0         Little Big Top          10.0    0.092100   2006-01-01
1  Dancer, Texas Pop. 81          10.0    0.376662   1998-05-01
2       Stiff Upper Lips          10.0    0.356495   1998-06-12
3  Me You and Five Bucks          10.0    0.094105   2015-07-07
4              Sardaarji           9.5    0.296981   2015-06-26


4803

In [7]:
print(df_genre_popularity.head())
len(df_genre_popularity)

  genre_name                                              title  popularity  \
0     Action                                           Deadpool  514.569956   
1     Action                            Guardians of the Galaxy  481.098624   
2     Action                                 Mad Max: Fury Road  434.278564   
3     Action                                     Jurassic World  418.708552   
4     Action  Pirates of the Caribbean: The Curse of the Bla...  271.972889   

   vote_average  
0           7.4  
1           7.9  
2           7.2  
3           6.5  
4           7.5  


12160

In [8]:
df_movie_characters_with_actors.head()

,movie_title,character_name,actor_name
0,Pirates of the Caribbean: At World's End,Captain Jack Sparrow,Johnny Depp
1,Pirates of the Caribbean: At World's End,Will Turner,Orlando Bloom
2,Pirates of the Caribbean: At World's End,Elizabeth Swann,Keira Knightley
3,Pirates of the Caribbean: At World's End,William Bootstrap Bill Turner,Stellan Skarsgu00e5rd
4,Pirates of the Caribbean: At World's End,Captain Sao Feng,Chow Yun-fat


In [9]:
df_movie_production_companies.head()


,movie_title,company_name
0,Four Rooms,Miramax Films
1,Four Rooms,A Band Apart
2,Star Wars,Lucasfilm
3,Star Wars,Twentieth Century Fox Film Corporation
4,Finding Nemo,Pixar Animation Studios


In [10]:
df_most_expensive_movies.head()

,title,budget
0,Pirates of the Caribbean: On Stranger Tides,380000000
1,Pirates of the Caribbean: At World's End,300000000
2,Avengers: Age of Ultron,280000000
3,Superman Returns,270000000
4,Tangled,260000000


In [11]:
df_available_languages_by_movie.head()

,movie_title,language_name,language_role
0,Four Rooms,English,Spoken
1,Star Wars,English,Spoken
2,Finding Nemo,English,Spoken
3,Forrest Gump,English,Spoken
4,American Beauty,English,Spoken


# Tratamento de dados
# Lidando com possiveis valores nulos

In [12]:

df_top_rated['vote_average'] = df_top_rated['vote_average'].fillna(df_top_rated['vote_average'].mean())
df_genre_popularity = df_genre_popularity.dropna(subset=['title'])

print(df_top_rated.head())
df_genre_popularity.head()

                   title  vote_average  popularity release_date
0         Little Big Top          10.0    0.092100   2006-01-01
1  Dancer, Texas Pop. 81          10.0    0.376662   1998-05-01
2       Stiff Upper Lips          10.0    0.356495   1998-06-12
3  Me You and Five Bucks          10.0    0.094105   2015-07-07
4              Sardaarji           9.5    0.296981   2015-06-26


,genre_name,title,popularity,vote_average
0,Action,Deadpool,514.569956,7.4
1,Action,Guardians of the Galaxy,481.098624,7.9
2,Action,Mad Max: Fury Road,434.278564,7.2
3,Action,Jurassic World,418.708552,6.5
4,Action,Pirates of the Caribbean: The Curse of the Bla...,271.972889,7.5


# Converter tipos de dados

In [13]:

df_top_rated['release_date'] = pd.to_datetime(df_top_rated['release_date'], errors='coerce')
df_top_rated['vote_average'] = pd.to_numeric(df_top_rated['vote_average'], errors='coerce')
df_top_rated['popularity'] = pd.to_numeric(df_top_rated['popularity'], errors='coerce')
df_country_rating['avg_rating'] = pd.to_numeric(df_country_rating['avg_rating'])
 

### Moeda

In [14]:
df_most_expensive_movies['budget'] = df_most_expensive_movies['budget'].apply(lambda x: f'R$ {x:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.'))

df_most_expensive_movies['budget']

0       R$ 380.000.000,00
1       R$ 300.000.000,00
2       R$ 280.000.000,00
3       R$ 270.000.000,00
4       R$ 260.000.000,00
              ...        
4798              R$ 0,00
4799              R$ 0,00
4800              R$ 0,00
4801              R$ 0,00
4802              R$ 0,00
Name: budget, Length: 4803, dtype: object

# Criar colunas derivadas

Cria uma nova coluna chamada release_year pegando só o ano da data de lançamento (release_date).
Se a data era 2021-07-16, agora você tem só 2021


Criando faixas (bins) de nota:

de 0 até 5 → 'Baixo'

de 5 até 7 → 'Médio'

de 7 até 10 → 'Alto'

Cria uma nova coluna chamada rating_category com base na nota (vote_average), usando as faixas acima.

Cada filme agora vai ter uma categoria de avaliação: 'Baixo', 'Médio' ou 'Alto'.


In [15]:
df_top_rated['release_year'] = df_top_rated['release_date'].dt.year
bins = [0, 5, 7, 10]
labels = ['Baixo', 'Médio', 'Alto']
df_top_rated['rating_category'] = pd.cut(df_top_rated['vote_average'], bins=bins, labels=labels, include_lowest=True)

df_top_rated.head()

,title,vote_average,popularity,release_date,release_year,rating_category
0,Little Big Top,10.0,0.092100,2006-01-01,2006.0,Alto
1,"Dancer, Texas Pop. 81",10.0,0.376662,1998-05-01,1998.0,Alto
2,Stiff Upper Lips,10.0,0.356495,1998-06-12,1998.0,Alto
3,Me You and Five Bucks,10.0,0.094105,2015-07-07,2015.0,Alto
4,Sardaarji,9.5,0.296981,2015-06-26,2015.0,Alto


# Arredondar valores

In [16]:
df_genre_popularity['vote_average'] = df_genre_popularity['vote_average'].round(2)
df_genre_popularity['popularity'] = df_genre_popularity['popularity'].round(2)
df_genre_popularity.head()

df_top_rated['popularity'] = df_top_rated['popularity'].round(2)

df_country_rating['avg_rating'] = df_country_rating['avg_rating'].round(2)
df_country_rating.head()


,country_name,avg_rating,movie_count
0,Austria,7.08,6
1,Argentina,6.99,9
2,Sweden,6.91,19
3,New Zealand,6.73,28
4,Iceland,6.72,6


# Agrupar e sumarizar

In [17]:
df_genre_summary = df_genre_popularity.groupby('genre_name').agg({
    'vote_average': 'mean',
    'popularity': 'mean',
    'title': 'count'
}).reset_index().rename(columns={'title': 'movie_count'})
df_genre_summary['vote_average'] = df_genre_summary['vote_average'].round(2)
df_genre_summary['popularity'] = df_genre_summary['popularity'].round(2)

df_genre_summary.head()


,genre_name,vote_average,popularity,movie_count
0,Action,5.99,30.94,1154
1,Adventure,6.16,39.27,790
2,Animation,6.34,38.81,234
3,Comedy,5.95,18.22,1722
4,Crime,6.27,22.85,696


## Função para criar uma tabela no Excel

Essa função cria realmente uma tabela no excel, não apenas coloca os dados numa aba de folha do excel. Isso irá facilitar o consumo dos dados no PowerBI


In [18]:
def write_table(writer, df, sheet_name, table_name):
    df.to_excel(writer, sheet_name=sheet_name, startrow=1, header=False, index=False)
    
    worksheet = writer.sheets[sheet_name]
    
    # Aqui ele cria um cabeçalho manual
    for col_num, value in enumerate(df.columns.values):
        worksheet.write(0, col_num, value)
    
    # Define o tamanho da tabela aqui:
    (max_row, max_col) = df.shape
    column_settings = [{'header': column} for column in df.columns]
    
    worksheet.add_table(0, 0, max_row, max_col - 1, {
        'columns': column_settings,
        'name': table_name,
        'style': 'Table Style Medium 9'
    })

# Exportar para um excel com varias páginas com cada DF

In [19]:
with pd.ExcelWriter('movies_data_report.xlsx', engine='xlsxwriter') as writer:
    write_table(writer, df_top_rated, 'Top Rated Movies', 'TopRatedMovies')
    write_table(writer, df_genre_popularity, 'Genre Popularity', 'GenrePopularity')
    write_table(writer, df_genre_summary, 'Genre Summary', 'GenreSummary')
    write_table(writer, df_country_rating, 'Country Rating', 'CountryRating')
    write_table(writer, df_movie_characters_with_actors, 'Characters & Actors', 'CharactersActors')
    write_table(writer, df_movie_production_companies, 'Production Companies', 'ProductionCompanies')
    write_table(writer, df_most_expensive_movies, 'Most Expensive Movies', 'MostExpensiveMovies')
    write_table(writer, df_available_languages_by_movie, 'Languages by Movie', 'LanguagesByMovie')